# Lightweight networks and MobileNet

We have seen that complex networks require significant computational resources, such as GPU, for training, and also for fast inference. However, it turns out that a model with significantly smaller number of parameters in most cases can still be trained to perform reasonably well. In other words, increase in the model complexity typically results in small (non-proportional) increase in the model performance.

We have observed this in the beginning of the module when training MNIST digit classification. The accuracy of simple dense model was not significantly worse than that of a powerful CNN. Increasing the number of CNN layers and/or number of neurons in the classifier allowed us to gain a few percents of accuracy at most.

This leads us to the idea that we can experiment with Lightweight network architectures in order to train faster models. This is especially important if we want to be able to execute our models on mobile devices.

This module will rely on the Cats and Dogs dataset that we have downloaded in the previous unit. First we will make sure that the dataset is available.

In [ ]:
!wget https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/computer-vision-pytorch/pytorchcv.py

In [1]:
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
from torchinfo import summary
import os

from pytorchcv import train, display_dataset, train_long, load_cats_dogs_dataset, validate, common_transform

> 因为wget命令在windows下不可用，所以我们直接下载数据集，然后解压到当前目录下的data文件夹中即可。

In [2]:
if not os.path.exists('data/kagglecatsanddogs_5340.zip'):
    !wget -P data -q https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag)
/anaconda/envs/py37_pytorch/lib/python3.7/site-packages/PIL/TiffImagePlugin

In [5]:
dataset, train_loader, test_loader = load_cats_dogs_dataset()

d:\anaconda\lib\site-packages\PIL\TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


## MobileNet

In the previous unit, we have seen **ResNet** architecture for image classification. More lightweight analog of ResNet is **MobileNet**, which uses so-called *Inverted Residual Blocks*. Let's load pre-trained mobilenet and see how it works:

In [2]:
print (torch.__version__)
print (torchvision.__version__)

1.12.0+cpu
0.13.0+cpu


In [3]:
model = torch.hub.load('pytorch/vision:v0.13.0', 'mobilenet_v2', weights='MobileNet_V2_Weights.DEFAULT')
model.eval()
print(model)

Using cache found in C:\Users\dongl/.cache\torch\hub\pytorch_vision_v0.13.0
Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to C:\Users\dongl/.cache\torch\hub\checkpoints\mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:01<00:00, 8.76MB/s]

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

Let's apply the model to our dataset and make sure that it works.

In [7]:
sample_image = dataset[0][0].unsqueeze(0)
res = model(sample_image)
print(res[0].argmax())

tensor(186)


The result (281) is the ImageNet class number, which we have talked about in the previous unit.

> Note that the number of parameters in MobileNet and full-scale ResNet model differ significantly. In some ways, MobileNet is more compact that VGG model family, which is less accurate. However, reduction in the number of parameters naturally leads to some drop in the model accuracy.



## Using MobileNet for transfer learning

Now let's perform the same transfer learning process as in previous unit, but using MobileNet. First of all, let's freeze all parameters of the model:

In [8]:
for x in model.parameters():
    x.requires_grad = False

Then, replace the final classifier. We also transfer the model to our default training device (GPU or CPU):

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.classifier = nn.Linear(1280,2)
model = model.to(device)
summary(model,input_size=(1,3,244,244))

Layer (type:depth-idx)                             Output Shape              Param #
MobileNetV2                                        [1, 2]                    --
├─Sequential: 1-1                                  [1, 1280, 8, 8]           --
│    └─Conv2dNormActivation: 2-1                   [1, 32, 122, 122]         --
│    │    └─Conv2d: 3-1                            [1, 32, 122, 122]         (864)
│    │    └─BatchNorm2d: 3-2                       [1, 32, 122, 122]         (64)
│    │    └─ReLU6: 3-3                             [1, 32, 122, 122]         --
│    └─InvertedResidual: 2-2                       [1, 16, 122, 122]         --
│    │    └─Sequential: 3-4                        [1, 16, 122, 122]         (896)
│    └─InvertedResidual: 2-3                       [1, 24, 61, 61]           --
│    │    └─Sequential: 3-5                        [1, 24, 61, 61]           (5,136)
│    └─InvertedResidual: 2-4                       [1, 24, 61, 61]           --
│    │    └─Sequential

Now let's do the actual training:


In [10]:
train_long(model,train_loader,test_loader,loss_fn=torch.nn.CrossEntropyLoss(),epochs=1,print_freq=90)

Epoch 0, minibatch 0: train acc = 0.5625, train loss = 0.020442266017198563
Epoch 0, minibatch 90: train acc = 0.9350961538461539, train loss = 0.005014345541105166
Epoch 0, minibatch 180: train acc = 0.9428522099447514, train loss = 0.004601399213569599
Epoch 0, minibatch 270: train acc = 0.9406134686346863, train loss = 0.005114285268466851
Epoch 0, minibatch 360: train acc = 0.9430401662049861, train loss = 0.0049250525450772525
Epoch 0, minibatch 450: train acc = 0.9430432372505543, train loss = 0.0049824926118364355
Epoch 0, minibatch 540: train acc = 0.9450092421441775, train loss = 0.00479926992476317
Epoch 0 done, validation acc = 0.9734, validation loss = 0.002392814064025879


> 对比用Tesla T4的GPU的colab笔记本训练效果3min，我们在本地用16核cpu运行需要22min44.2s

## Takeaway

Notice that MobileNet results in almost the same accuracy as VGG-16, and just slightly lower than full-scale ResNet. 

The main advantage of small models, such as MobileNet or ResNet-18 is that they can be used on mobile devices. [Here](https://pytorch.org/mobile/android/) is official example of using ResNet-18 on Android device, and [here](https://heartbeat.fritz.ai/pytorch-mobile-image-classification-on-android-5c0cfb774c5b) is similar example using MobileNet. 